In [1]:
import os 
import json 
import string
import polars as pl 
import openpyxl as xl 
import shutil 
from datetime import datetime as dt 
here = os.path.abspath('')
### set downloaded files directory: 
# files_dir = os.path.join(here,'YearlyData')
files_dir = os.path.join(here, 'MonthlyData')

### while parsing excel files - if the same county-month have been downloaded multiple times 
# it will not be duplicated in the final data set
# duplicate files will be copied to this directory:
dupe_files_dir = os.path.join(files_dir, 'Duplicates')
### set output file destination 
output = os.path.join(here, f'SRSAnnualReturnAConsolidatedMonthlyData.xlsx')
# output = os.path.join(here, f'SRSAnnualReturnAConsolidatedYearlyData.xlsx')
print(dt.now())


2025-10-20 07:39:31.410744


In [2]:
files_to_read = []
for root, dirs, files in os.walk(files_dir):
    for fn, file in enumerate(files):
        if file.endswith('.xlsx') and not file.startswith('~'): # and fn < 1:
            file_path = os.path.join(root, file)
            files_to_read.append(file_path)
print(len(files_to_read), files_to_read)

1340 ['/Users/danwelsh/Projects/CrimeData/PA_UCR_WebScraper/AnnualSRSSummaryReport/ReturnA/MonthlyData/SRS_AnnualSummary_ReturnA - 2025-10-19T182251.464.xlsx', '/Users/danwelsh/Projects/CrimeData/PA_UCR_WebScraper/AnnualSRSSummaryReport/ReturnA/MonthlyData/SRS_AnnualSummary_ReturnA - 2025-10-19T120852.398.xlsx', '/Users/danwelsh/Projects/CrimeData/PA_UCR_WebScraper/AnnualSRSSummaryReport/ReturnA/MonthlyData/SRS_AnnualSummary_ReturnA - 2025-10-18T165517.067.xlsx', '/Users/danwelsh/Projects/CrimeData/PA_UCR_WebScraper/AnnualSRSSummaryReport/ReturnA/MonthlyData/SRS_AnnualSummary_ReturnA - 2025-10-18T164849.595.xlsx', '/Users/danwelsh/Projects/CrimeData/PA_UCR_WebScraper/AnnualSRSSummaryReport/ReturnA/MonthlyData/SRS_AnnualSummary_ReturnA - 2025-10-18T135346.244.xlsx', '/Users/danwelsh/Projects/CrimeData/PA_UCR_WebScraper/AnnualSRSSummaryReport/ReturnA/MonthlyData/SRS_AnnualSummary_ReturnA - 2025-10-18T181749.036.xlsx', '/Users/danwelsh/Projects/CrimeData/PA_UCR_WebScraper/AnnualSRSSummary

In [3]:
### specific to the exported structure/format of the SRS Summary Report - Return A report when exported to Excel 
### Consolidate the individual county files into one dataset:
### read each file into a dataframe, add the frame to a list of frames, concatenate all frames into 1
file_frames = []
# keeps track of county-time period files already loaded to manage duplicates
load_keys = [] 
for fn, file_path in enumerate(files_to_read):
    if os.path.exists(file_path):
        ### load workbook and worksheet
        wb = xl.load_workbook(file_path)
        ws = wb.worksheets[0]
        ### 
        begin = ws['d8'].value
        begin_dt = begin[-10:]
        end = ws['h9'].value
        end_dt = end[-10:]
        county = ws['b16'].value
        county_name = county.replace("County: ",'')
        # get printed on value from top right of report... report run date 
        report_run_date = ws['m3'].value
        ### actual data table starts on b18 
        table_start = 'b18'
        load_key = f'{county}_{begin_dt}_{end_dt}'
        if load_key not in load_keys:
            # read the table starting with header row on row 17 (0 based index, Excel row 18)
            df = pl.read_excel(file_path, read_options={'header_row':17}, infer_schema_length=0)
            cols = df.columns 
            # manually add columns from the parameters above the table; 
            df = df.with_columns(
                pl.lit(report_run_date).alias('ReportRunDate'),
                pl.lit(file_path).alias('SourceFile'),
                pl.lit(begin_dt ).alias('BeginDate'),
                pl.lit(end_dt).alias('EndDate'),
                pl.lit(county_name).alias('County')
            )
            # Re-order columns for cleanliness
            df = df.with_row_index(name='ClassificationSortOrder',offset=1)
            order_cols = ['County','BeginDate','EndDate','ClassificationSortOrder'] + cols + ['ReportRunDate','SourceFile']
            df = df.select(order_cols)
            file_frames.append(df)
            print(fn, '/', len(files_to_read), begin_dt, end_dt, county_name, file)
            load_keys.append(load_key)
        else:
            #move duplicate file and dont load:
            move_dupe_file_to = os.path.join(dupe_files_dir, file)
            shutil.move(file_path, move_dupe_file_to)
            print(f'DUPLICATE: {load_key} moved to {move_dupe_file_to}')


consolidated_df = pl.concat(file_frames)
print(consolidated_df.shape)
### distinct count of counties by begin date - validate all 67 are loaded for each time period
with pl.Config(set_tbl_rows=48):
    print(consolidated_df.group_by(['BeginDate']).agg(pl.col('County').n_unique()).sort('BeginDate'))
print(dt.now().isoformat())

0 / 1340 09/01/2024 09/30/2024 Clarion County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
1 / 1340 02/01/2024 02/29/2024 Bedford County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
2 / 1340 08/01/2025 08/31/2025 Armstrong County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
3 / 1340 07/01/2025 07/31/2025 Wayne County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
4 / 1340 04/01/2025 04/30/2025 Forest County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
5 / 1340 09/01/2025 09/30/2025 Luzerne County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
6 / 1340 04/01/2024 04/30/2024 Westmoreland County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
7 / 1340 01/01/2025 01/31/2025 Wayne County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
8 / 1340 01/01/2025 01/31/2025 Franklin County SRS_AnnualSummary_ReturnA - 2025-10-18T125657.345.xlsx
9 / 1340 12/01/2024 12/31/2024 Washington County SRS_AnnualSummary_ReturnA - 2025-10-18T

In [4]:
# consolidated_df = pl.read_excel(os.path.join(here, f'SRSAnnualReturnAConsolidated.xlsx'))

### rename the default spreadsheet column names (keep in order, removes line breaks in some column names)
consolidated_columns_rename = [
    'County',
    'BeginDate',
    'EndDate',
    'ClassificationSortOrder',
    'Classification of Offenses',
    'Offenses Reported',
    'Unfounded',
    'Actual Offenses',
    'Total Offenses Cleared',
    'Clearances Involving Persons Under 18 Yr. of Age',
    'ReportRunDate',
    'SourceFile',
]
consolidated_df.columns = consolidated_columns_rename

### get county details sheet from ClassificationOfOffenses spreadsheet - pulling in county_fips_code for joining to census data
## (used as a generic metadata/lookup/supplemental spreadsheet for this process - kept separate from the main consolidated SRS data file)
lookup_xl = os.path.join(here, 'ClassificationOfOffenses.xlsx')
county_df = pl.read_excel(lookup_xl, sheet_name='Counties')
county_df = county_df.select(['County','county_fips_code'])
county_df.columns = ['County Name','County FIPS Code']
### merge county data with main data file to include county FIPS code with the main data - used to link up with census data 
consolidated_df = consolidated_df.join(county_df, left_on='County', right_on='County Name', how='left')

consolidated_columns_after_join = [
    'County',
    'County FIPS Code',
    'BeginDate',
    'EndDate',
    'ClassificationSortOrder',
    'Classification of Offenses',
    'Offenses Reported',
    'Unfounded',
    'Actual Offenses',
    'Total Offenses Cleared',
    'Clearances Involving Persons Under 18 Yr. of Age',
    'ReportRunDate',
    'SourceFile',
]
consolidated_df = consolidated_df.select(consolidated_columns_after_join)

### write excel with county fips code 

consolidated_df.write_excel(output)

print(dt.now())

2025-10-20 07:46:28.651289


In [5]:
# df = df.with_row_index(name='SortOrder',offset=1)
with pl.Config(set_tbl_cols=12, set_tbl_width_chars=200, set_tbl_rows=500):
    print(df)




shape: (59, 12)
┌─────────────────┬────────────┬────────────┬─────────────────────┬─────────────────────┬───────────┬───────────┬──────────┬────────────────┬─────────────────────┬───────────────┬────────────────────┐
│ County          ┆ BeginDate  ┆ EndDate    ┆ ClassificationSortO ┆ Classification of   ┆ Offenses  ┆ Unfounded ┆ Actual   ┆ Tot. Offenses  ┆ Clearances          ┆ ReportRunDate ┆ SourceFile         │
│ ---             ┆ ---        ┆ ---        ┆ rder                ┆ Offenses            ┆ Reported  ┆ ---       ┆ Offenses ┆ Cleared        ┆ Involving Persons   ┆ ---           ┆ ---                │
│ str             ┆ str        ┆ str        ┆ ---                 ┆ ---                 ┆ ---       ┆ str       ┆ ---      ┆ ---            ┆ …                   ┆ str           ┆ str                │
│                 ┆            ┆            ┆ u32                 ┆ str                 ┆ str       ┆           ┆ str      ┆ str            ┆ ---                 ┆               ┆ 